# Generate time-varying surface forcing

### Code to generate ONLY time-varying surface forcing based on ERA5 and backward trajectory and high-res sea ice satellite data from Univ. of Bremen for COMBLE LES/SCM intercomparison
### Developed by Tim Juliano at NCAR/RAL originally on 8/26/22
### This file generated for the ARM Data Archive on 2/1/23
### We have the option of starting as far as 28 hours before arriving at Andenes (03/12/20 at 14 UTC)
### Includes lat/lon correction by Peng Wu (PNNL)

### Import libraries

In [39]:
import netCDF4
import numpy as np
import numpy.ma as ma
import sys
from netCDF4 import Dataset, date2num,num2date
import datetime as dt
import os
import matplotlib.pyplot as plt
from scipy import interpolate
import geopy.distance
from scipy.ndimage import gaussian_filter1d
#import gdal
import copy

### !!! BEGIN USER MODS !!!
#### How many hours before ice edge do you want to start?
#### Note: t0_h = 0 means you are starting approx. at ice edge, t0_h = 10 means you are starting 10 h upstream (north) of the ice edge
#### Note: t0_h must be an integer

In [40]:
t0_h = 10
if t0_h < 0 or t0_h > 10:
    sys.exit('Error: Please set 0 <= t0_h >= 10')
    
verbose = False # switch to output additional info

### !!! END USER MODS !!!

### Set some things

In [41]:
nhrs = 18 + t0_h + 1 # total number of simulation hours, including t0; default is from 2020-03-12 at 14 UTC to 2020-03-14 at 00 UTC
if t0_h == 0:
    start_time = '2020-03-13 00:00:00'
    start_day = 13
    start_hour = 0
else:
    start_time = '2020-03-12 ' + str(24-t0_h) + ':00:00'
    start_day = 12
    start_hour = 24-t0_h
print ('Start time is: ' + start_time)

Start time is: 2020-03-12 14:00:00


### Forcing NetCDF name and delete file if already exists

In [42]:
savename = 'COMBLE_INTERCOMPARISON_SFC_FORCING.nc'

if os.path.exists(savename):
    os.remove(savename)
    print('The file ' + savename + ' has been deleted successfully')

The file COMBLE_INTERCOMPARISON_SFC_FORCING.nc has been deleted successfully


### Get LES domain locations

In [43]:
fname = 'LES_domain_location_28h_18Z_Mar13_2020.txt'

les_loc = np.loadtxt(fname,skiprows=1)
les_hh = les_loc[:,0]
les_hh_idx = np.where(les_hh>=-1.*t0_h)[0]
les_lat = les_loc[les_hh_idx,1]
les_lon = les_loc[les_hh_idx,2]

les_lat_mid = round(np.mean(les_lat),1)
les_lon_mid = round(np.mean(les_lon),1)
print ('LES domain mid point: ' + 'lat=' + str(les_lat_mid) + 'N, lon=' + str(les_lon_mid) + 'E')

LES domain mid point: lat=77.6N, lon=8.6E


### Calculate sea ice concentration along trajectory

In [44]:
sic_file = netCDF4.Dataset('Svalbard_asi-AMSR2-n10m-20200313_m.nc')
sic = sic_file.variables['z'][:,:]
sic_lat = sic_file.variables['lat'][:]
sic_lon = sic_file.variables['lon'][:]

# Read high-res domain location file
les_loc_hr = np.loadtxt(fname,skiprows=1)
les_hh_hr = les_loc_hr[:,0]
les_hh_hr_idx = np.where(les_hh_hr>=-1.*t0_h)[0]
les_lat_hr = les_loc_hr[les_hh_hr_idx,1]
les_lon_hr = les_loc_hr[les_hh_hr_idx,2]

sic_traj = np.empty(len(les_hh_hr_idx))
for i in np.arange(len(les_hh_hr_idx)):
    abslat = np.abs(sic_lat-les_lat_hr[i])
    abslon = np.abs(sic_lon-les_lon_hr[i])
    jlat = np.argmin(abslat)
    ilon = np.argmin(abslon)
    sic_traj[i] = sic[jlat,ilon]

### Read in data from ERA5 backtrajectory (netCDF)

In [45]:
# File name
fname = 'theta_temp_rh_sh_uvw_sst_along_trajectory_era5ml_28h_end_2020-03-13-18.nc'

# Open dataset
dataset = netCDF4.Dataset(fname, "r")

# Read variables (1D arrays are time, 2D arrays are time x pressure level)
hours = dataset.variables['Time'][:]
lat = dataset.variables['Latitude'][:]
lon = dataset.variables['Longitude'][:]
sst = dataset.variables['SST'][:]

### Reverse time dimension of sst, so that beginning of backward trajectory is in first position

In [46]:
sst = sst[::-1]

### Unmask sst field and get index according to t0_h (furthest north we can go is 28h after backtrajectory initialization from Andenes, or 3/12/20 at 14 UTC)

In [47]:
sst_real = ma.getdata(sst)
loopidx = np.where(sst_real>0.0)[0]
t0 = loopidx[0]

### Get time series information for sfc forcing

In [48]:
# Interpolate SST
tmp_hh = np.arange(-10,19,1) # these are the hours we have SST data from the ERA5 backward trajectory file
f = interpolate.interp1d(tmp_hh, sst[t0:])
sst_interp = f(les_hh_hr[les_hh_hr_idx])

# Modification for over ice/MIZ
tsk_ice = 247.0
sst_ts = np.empty(len(sic_traj))
for i in np.arange(len(sic_traj)):
    if sic_traj[i] > 90.0: # over ice
        sst_ts[i] = tsk_ice
    elif sic_traj[i] > 0.0: # MIZ
        sst_ts[i] = (sic_traj[i]/100.)*tsk_ice + (1.-(sic_traj[i]/100.))*sst_interp[i] # MIZ
    else: # open ocean
        sst_ts[i] = sst_interp[i]

    if verbose == True:
        print ('Computed TSK for hour ' + str(round(5*i/60.,3)) + ': '
               + str(round(sst_ts[i],3)) + ' with SIC = ' + str(sic_traj[i]/100.) +
               ' and SST = ' + str(round(sst_interp[i],3)))

### Create new netcdf file

In [49]:
try: ncfile.close()  # just to be safe, make sure dataset is not already open.
except: pass
ncfile = Dataset('./' + savename,mode='w',format='NETCDF3_CLASSIC') 

### Create dimensions

In [50]:
time_dim = ncfile.createDimension('time', None)    # unlimited axis (can be appended to)

### Create global attributes

In [51]:
ncfile.title='Time-varying surface conditions for 13 March 2020 COMBLE intercomparison case'
ncfile.reference='https://arm-development.github.io/comble-mip/'
ncfile.authors='Timothy W. Juliano (NCAR/RAL, tjuliano@ucar.edu); Florian Tornow (NASA/GISS, ft2544@columbia.edu); Ann M. Fridlind (NASA/GISS, ann.fridlind@nasa.gov)'
ncfile.version='Created on 2023-02-01'
ncfile.format_version='DEPHY SCM format version 1'
ncfile.script='create_comble_sfc_forcing_era5ml.ipynb'
ncfile.startDate=start_time
ncfile.endDate='2020-03-13 18:00:00'
ncfile.force_geo=1
ncfile.surfaceType='ocean'
ncfile.surfaceForcing='ts'
ncfile.lat=str(les_lat_mid) + ' deg N'
ncfile.dx='150 m'
ncfile.dy='150 m'
ncfile.dz='see zw_grid variable'
ncfile.nx='648'
ncfile.ny='648'
ncfile.nz='160'

### Create variables

#### Dimensions

In [52]:
time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'seconds since ' + start_time
time.long_name = 'time'

#### Time-varying forcing - surface

In [53]:
ts = ncfile.createVariable('ts', np.float64, ('time',))
ts.units = 'K'
ts.long_name = 'surface temperature'

### Write data

In [54]:
ts[:] = sst_ts

### Add times

In [55]:
dates = []
nmin = int((18 + t0_h)*12 + 1)
curr_hh = copy.deepcopy(start_hour)
curr_dd = copy.deepcopy(start_day)
ii = 0
first_pass = 1
for i in np.arange(nmin):
    curr_min = 5*ii
    if curr_min % 60 == 0 and first_pass == 0:
        curr_hh+=1
        ii = 0
        curr_min = 5*ii
        if curr_hh == 24:
            curr_hh = 0
            curr_dd+=1
    
    dates.append(dt.datetime(2020,3,curr_dd,curr_hh,curr_min))
    
    first_pass = 0
    ii+=1

times = date2num(dates, time.units)
time[:] = times

### Close the file

In [56]:
# first print the Dataset object to see what we've got
print(ncfile)
# close the Dataset.
ncfile.close(); print('Dataset is closed!')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    title: Time-varying surface conditions for 13 March 2020 COMBLE intercomparison case
    reference: https://arm-development.github.io/comble-mip/
    authors: Timothy W. Juliano (NCAR/RAL, tjuliano@ucar.edu); Florian Tornow (NASA/GISS, ft2544@columbia.edu); Ann M. Fridlind (NASA/GISS, ann.fridlind@nasa.gov)
    version: Created on 2023-02-01
    format_version: DEPHY SCM format version 1
    script: create_comble_sfc_forcing_era5ml.ipynb
    startDate: 2020-03-12 14:00:00
    endDate: 2020-03-13 18:00:00
    force_geo: 1
    surfaceType: ocean
    surfaceForcing: ts
    lat: 77.6 deg N
    dx: 150 m
    dy: 150 m
    dz: see zw_grid variable
    nx: 648
    ny: 648
    nz: 160
    dimensions(sizes): time(337)
    variables(dimensions): float64 time(time), float64 ts(time)
    groups: 
Dataset is closed!
